In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tourney_results = pd.read_csv("TourneyCompactResults.csv", header=0, \
                    quoting=3)
season_results = pd.read_csv("RegularSeasonCompactResults.csv", header=0, \
                    quoting=3)
seeds = pd.read_csv("TourneySeeds.csv", header=0, \
                    quoting=3)
pd.options.mode.chained_assignment = None  #Stop annoying warnings

In [ ]:
year=2003
df_season=season_results[season_results['Season']==year]
df_tourney=tourney_results[tourney_results['Season']==year]
df_seeds=seeds[seeds['Season']==year]

In [ ]:
def seed_to_int(seed):
    """Get just the digits from the seeding. Return as int"""
    s_int = int(seed[1:3])
    return s_int
df_seeds['n_seed'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label

In [ ]:
win_lose=df_season[['Wteam','Lteam']]
test=pd.DataFrame()
test['LeftTeam']=win_lose['Wteam']
test['RightTeam']=win_lose['Wteam']
test['WinValue']=0

In [ ]:
for index,row in win_lose.iterrows():
    i=0
    if row['Wteam']>row['Lteam']:
        i=1
    if i==1:
        test['LeftTeam'][index]=row['Wteam']
        test['RightTeam'][index]=row['Lteam']
        test['WinValue'][index]=i
    else:
        test['LeftTeam'][index]=row['Lteam']
        test['RightTeam'][index]=row['Wteam']

In [ ]:
season_compiled=test.merge(df_seeds,left_on='LeftTeam',right_on='Team').merge(df_seeds,left_on='RightTeam',right_on='Team')
season_compiled['seed_diff']=season_compiled['n_seed_x']-season_compiled['n_seed_y']
season_compiled=season_compiled[['seed_diff','WinValue']]

In [ ]:
win_lose=df_tourney[['Wteam','Lteam']]
test=pd.DataFrame()
test['LeftTeam']=win_lose['Wteam']
test['RightTeam']=win_lose['Wteam']
test['WinValue']=0

In [ ]:
for index,row in win_lose.iterrows():
    i=0
    if row['Wteam']>row['Lteam']:
        i=1
    if i==1:
        test['LeftTeam'][index]=row['Wteam']
        test['RightTeam'][index]=row['Lteam']
        test['WinValue'][index]=i
    else:
        test['LeftTeam'][index]=row['Lteam']
        test['RightTeam'][index]=row['Wteam']

In [ ]:
tourney_compiled=test.merge(df_seeds,left_on='LeftTeam',right_on='Team').merge(df_seeds,left_on='RightTeam',right_on='Team')
tourney_compiled['seed_diff']=tourney_compiled['n_seed_x']-tourney_compiled['n_seed_y']
tourney_compiled=tourney_compiled[['seed_diff','WinValue']]

In [ ]:
df_train=season_compiled.iloc[:,0].values.reshape(-1,1)
w_train=season_compiled.iloc[:,1]
df_test=tourney_compiled.iloc[:,0].values.reshape(-1,1)
w_test=tourney_compiled.iloc[:,1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 10) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( df_train, w_train)

In [ ]:
result = forest.predict(df_test)

In [ ]:
sum(result==w_test)/result.size

In [ ]:
forest.predict_proba(df_test)

In [ ]:
from sklearn.svm import SVC
clf=SVC(probability=True)
clf.fit(df_train,w_train)

In [ ]:
result = clf.predict(df_test)

In [ ]:
sum(result==w_test)/result.size